In [13]:
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize
import os
import glob

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
from sklearn.feature_extraction.text import CountVectorizer

In [90]:
faults_diagnostics = pd.read_csv('faults_diagnostics.csv')

In [133]:
faults_diagnostics['DistanceLtd']

0          423178.70000
1          368419.30000
2          368419.30000
3          368419.30000
4          366722.90000
               ...     
1054108    369586.50000
1054109    423937.90000
1054110    465925.40000
1054111     28606.65625
1054112    368419.30000
Name: DistanceLtd, Length: 1054113, dtype: float64

In [92]:
faults_diagnostics.drop('Unnamed: 0', axis=1)

,FaultId,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,...,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,0.0,14.2100,False,66.48672,423178.70000,100.4,11.0,0.00,96.74375,...,0,111,17,True,2.0,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,2,9.2,14.3550,unknown,NaN,368419.30000,181.4,18.0,36.54,201.08750,...,11,629,12,True,127.0,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000
2,3,9.2,14.3550,unknown,NaN,368419.30000,181.4,18.0,36.54,201.08750,...,11,1807,2,False,127.0,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000
3,4,9.2,14.3550,unknown,NaN,368419.30000,181.4,18.0,36.54,201.08750,...,11,1807,2,True,127.0,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000
4,5,17.6,3.5525,unknown,NaN,366722.90000,177.8,28.5,45.82,225.78130,...,0,4364,17,False,2.0,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1054108,1248454,25.2,14.3550,unknown,NaN,369586.50000,185.0,28.0,36.54,208.45630,...,0,1761,17,False,3.0,2282,105439740,37.094768,-85.897407,2020-03-06 14:00:21.000
1054109,1248455,100.0,14.5000,True,64.62260,423937.90000,185.0,51.0,37.12,211.49370,...,0,1569,31,True,5.0,1994,105354084,34.390740,-79.461805,2020-03-06 14:04:59.000
1054110,1248456,0.0,14.3550,True,66.48672,465925.40000,186.8,62.0,41.18,212.84380,...,0,3216,10,True,1.0,1850,105336308,34.430370,-84.920509,2020-03-06 14:14:14.000
1054111,1248457,1.6,14.4275,False,67.72946,28606.65625,181.4,0.0,27.26,221.73120,...,0,111,18,True,8.0,2377,108605700,35.030925,-85.321527,2020-03-06 14:14:49.000


In [93]:
faults_diagnostics['EventTimeStamp'] = pd.to_datetime(faults_diagnostics['EventTimeStamp'])

In [130]:
faults_diagnostics

,Unnamed: 0,FaultId,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,...,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,0,1,0.0,14.2100,False,66.48672,423178.70000,100.4,11.0,0.00,...,0,111,17,True,2.0,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,1,2,9.2,14.3550,unknown,NaN,368419.30000,181.4,18.0,36.54,...,11,629,12,True,127.0,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000
2,2,3,9.2,14.3550,unknown,NaN,368419.30000,181.4,18.0,36.54,...,11,1807,2,False,127.0,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000
3,3,4,9.2,14.3550,unknown,NaN,368419.30000,181.4,18.0,36.54,...,11,1807,2,True,127.0,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000
4,4,5,17.6,3.5525,unknown,NaN,366722.90000,177.8,28.5,45.82,...,0,4364,17,False,2.0,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1054108,1054108,1248454,25.2,14.3550,unknown,NaN,369586.50000,185.0,28.0,36.54,...,0,1761,17,False,3.0,2282,105439740,37.094768,-85.897407,2020-03-06 14:00:21.000
1054109,1054109,1248455,100.0,14.5000,True,64.62260,423937.90000,185.0,51.0,37.12,...,0,1569,31,True,5.0,1994,105354084,34.390740,-79.461805,2020-03-06 14:04:59.000
1054110,1054110,1248456,0.0,14.3550,True,66.48672,465925.40000,186.8,62.0,41.18,...,0,3216,10,True,1.0,1850,105336308,34.430370,-84.920509,2020-03-06 14:14:14.000
1054111,1054111,1248457,1.6,14.4275,False,67.72946,28606.65625,181.4,0.0,27.26,...,0,111,18,True,8.0,2377,108605700,35.030925,-85.321527,2020-03-06 14:14:49.000


In [110]:
fault_test = faults_diagnostics[['EquipmentID', 'spn', 'fmi', 'EventTimeStamp']].sort_values('EventTimeStamp', ascending = False)

In [111]:
fault_test.reset_index(drop = True)

,EquipmentID,spn,fmi,EventTimeStamp
0,2015,829,3,2000-03-18 19:14:10
1,2015,96,3,2000-03-18 19:14:10
2,2015,829,3,2000-03-18 19:20:47
3,2015,96,3,2000-03-18 19:20:47
4,1849,792,7,2000-03-19 02:59:58
...,...,...,...,...
1054108,2282,1761,17,2020-03-06 14:00:26
1054109,1994,1569,31,2020-03-06 14:04:23
1054110,1850,3216,10,2020-03-06 14:13:38
1054111,2377,111,18,2020-03-06 14:14:13


In [132]:
fault_test.groupby('EquipmentID').rolling('1D', on = 'EventTimeStamp').count()

fmi  spn      EventTimeStamp
EquipmentID                                      
301         43654    1.0  1.0 2015-05-11 13:11:20
            45401    1.0  1.0 2015-05-13 08:22:32
            49950    1.0  1.0 2015-05-18 09:34:05
            53988    1.0  1.0 2015-05-21 13:57:35
            54080    2.0  2.0 2015-05-21 14:54:32
...                  ...  ...                 ...
2377        1054111  1.0  1.0 2020-03-06 14:14:13
            1054112  2.0  2.0 2020-03-06 14:15:34
2380        1053453  1.0  1.0 2020-03-04 10:06:10
2381        1054077  1.0  1.0 2020-03-06 11:56:38
            1054078  2.0  2.0 2020-03-06 11:58:09

[1054113 rows x 3 columns]

In [124]:
fault_test.rolling('1D', on = 'EventTimeStamp').count()

,EquipmentID,fmi,spn,EventTimeStamp
1025025,1.0,1.0,1.0,2000-03-18 19:14:10
1025026,2.0,2.0,2.0,2000-03-18 19:14:10
1025028,3.0,3.0,3.0,2000-03-18 19:20:47
1025027,4.0,4.0,4.0,2000-03-18 19:20:47
1025029,5.0,5.0,5.0,2000-03-19 02:59:58
...,...,...,...,...
1054108,288.0,288.0,288.0,2020-03-06 14:00:26
1054109,287.0,287.0,287.0,2020-03-06 14:04:23
1054110,284.0,284.0,284.0,2020-03-06 14:13:38
1054111,285.0,285.0,285.0,2020-03-06 14:14:13


In [ ]:
fault_test.rolling()

In [19]:
faults_diagnostics['Days'] = (faults_diagnostics['EventTimeStamp'] - faults_diagnostics['EventTimeStamp'].min()).dt.days

In [23]:
faults_diagnostics['Days'] = faults_diagnostics['Days'] - faults_diagnostics['Days'].min() + 1

In [8]:
faults_diagnostics.drop_duplicates(subset=['EquipmentID', 'spn', 'fmi', 'EventTimeStamp'], keep='first', inplace=True)

In [24]:
faults_diagnostics

,Unnamed: 0,FaultId,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,...,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,Days
0,0,1,0.0,14.2100,False,66.48672,423178.70000,100.4,11.0,0.00,...,111,17,True,2.0,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000,5453
1,1,2,9.2,14.3550,unknown,NaN,368419.30000,181.4,18.0,36.54,...,629,12,True,127.0,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000,5453
2,2,3,9.2,14.3550,unknown,NaN,368419.30000,181.4,18.0,36.54,...,1807,2,False,127.0,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000,5453
3,3,4,9.2,14.3550,unknown,NaN,368419.30000,181.4,18.0,36.54,...,1807,2,True,127.0,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000,5453
4,4,5,17.6,3.5525,unknown,NaN,366722.90000,177.8,28.5,45.82,...,4364,17,False,2.0,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000,5453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1054108,1054108,1248454,25.2,14.3550,unknown,NaN,369586.50000,185.0,28.0,36.54,...,1761,17,False,3.0,2282,105439740,37.094768,-85.897407,2020-03-06 14:00:21.000,7293
1054109,1054109,1248455,100.0,14.5000,True,64.62260,423937.90000,185.0,51.0,37.12,...,1569,31,True,5.0,1994,105354084,34.390740,-79.461805,2020-03-06 14:04:59.000,7293
1054110,1054110,1248456,0.0,14.3550,True,66.48672,465925.40000,186.8,62.0,41.18,...,3216,10,True,1.0,1850,105336308,34.430370,-84.920509,2020-03-06 14:14:14.000,7293
1054111,1054111,1248457,1.6,14.4275,False,67.72946,28606.65625,181.4,0.0,27.26,...,111,18,True,8.0,2377,108605700,35.030925,-85.321527,2020-03-06 14:14:49.000,7293


In [39]:
faults_diagnostics.drop('Unnamed: 0', axis=1)

,FaultId,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,...,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,Days
0,1,0.0,14.2100,False,66.48672,423178.70000,100.4,11.0,0.00,96.74375,...,111,17,True,2.0,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000,5453
1,2,9.2,14.3550,unknown,NaN,368419.30000,181.4,18.0,36.54,201.08750,...,629,12,True,127.0,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000,5453
2,3,9.2,14.3550,unknown,NaN,368419.30000,181.4,18.0,36.54,201.08750,...,1807,2,False,127.0,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000,5453
3,4,9.2,14.3550,unknown,NaN,368419.30000,181.4,18.0,36.54,201.08750,...,1807,2,True,127.0,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000,5453
4,5,17.6,3.5525,unknown,NaN,366722.90000,177.8,28.5,45.82,225.78130,...,4364,17,False,2.0,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000,5453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1054108,1248454,25.2,14.3550,unknown,NaN,369586.50000,185.0,28.0,36.54,208.45630,...,1761,17,False,3.0,2282,105439740,37.094768,-85.897407,2020-03-06 14:00:21.000,7293
1054109,1248455,100.0,14.5000,True,64.62260,423937.90000,185.0,51.0,37.12,211.49370,...,1569,31,True,5.0,1994,105354084,34.390740,-79.461805,2020-03-06 14:04:59.000,7293
1054110,1248456,0.0,14.3550,True,66.48672,465925.40000,186.8,62.0,41.18,212.84380,...,3216,10,True,1.0,1850,105336308,34.430370,-84.920509,2020-03-06 14:14:14.000,7293
1054111,1248457,1.6,14.4275,False,67.72946,28606.65625,181.4,0.0,27.26,221.73120,...,111,18,True,8.0,2377,108605700,35.030925,-85.321527,2020-03-06 14:14:49.000,7293


In [47]:
faults_diagnostics = faults_diagnostics.sort_values('Days', ascending = True).drop('Unnamed: 0', axis=1)

In [85]:
counts = faults_diagnostics.groupby(['Days', 'EquipmentID','spn', 'fmi']).size()

In [86]:
counts = pd.DataFrame(counts)

In [87]:
counts = counts.reset_index()

In [88]:
counts.rename({'0':'Count_of_Ocurrence'})

,Days,EquipmentID,spn,fmi,0
0,1,1826,794,2,1
1,1,1826,794,10,1
2,1,1849,792,7,2
3,1,1887,96,3,2
4,1,1887,829,3,2
...,...,...,...,...,...
226402,7293,2337,2863,7,2
226403,7293,2344,2863,7,2
226404,7293,2362,37,18,2
226405,7293,2377,111,18,2


In [89]:
counts[counts['spn'] == 5246].sort_values('Days', ascending = True)

,Days,EquipmentID,spn,fmi,0
95,3941,1452,5246,0,2
130,3941,1751,5246,0,1
160,3941,2175,5246,0,1
161,3941,2175,5246,16,1
581,5455,1487,5246,0,1
...,...,...,...,...,...
223653,7261,2211,5246,16,1
223883,7264,1854,5246,0,2
224550,7271,1872,5246,0,2
224664,7272,302,5246,19,1


In [ ]:
faults_diagnostics